In [33]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices,ComplementrySubjects,SubjectGroups
from Manage.serializers import PermanentSubjectSerializer
from StakeHolders.serializers import StudentSerializer
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string
from django.db.models import Count, Q

In [34]:
semester_obj = Semester.objects.get(slug='914150_1731255589')

In [37]:
subject_groups_qs = SubjectGroups.objects.filter(semester=semester_obj)

In [104]:
subject_groups =  {}
# subject_group_students = {}
total_hours = {}
for subject_group in subject_groups_qs:
    subjects = subject_group.subjects.all()
    subject_groups[tuple(subjects)] = subject_group.students.all()
    # subject_group_students[tuple(subjects)] = subject_group.students.all()
    for subject in subjects:
        if subject not in total_hours:
            total_hours[subject] = subject.subject_map.L + subject.subject_map.T + subject.subject_map.P

In [110]:
subject_groups =  {}
# subject_group_students = {}
total_hours = {}
for subject_group in subject_groups_qs:
    subjects = subject_group.subjects.all()
    subject_groups[tuple(subjects)] = subject_group.students.all()
    # subject_group_students[tuple(subjects)] = subject_group.students.all()
    for subject in subjects:
        if subject not in total_hours:
            total_hours[subject] = subject.subject_map.L + subject.subject_map.T + subject.subject_map.P
# Constraints
min_students = 0  # No minimum size for divisions
max_students = 90  # Maximum size per division

def allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students):
    # Step 1: Preprocess subject groups to split any group exceeding max_students
    preprocessed_groups = []
    for group, students in subject_groups.items():
        student_count = len(students)
        while student_count > max_students:            
            preprocessed_groups.append((group, students[:max_students]))
            subject_groups[group] = subject_groups[group][max_students:]
            student_count -= max_students
        if student_count > 0:
            preprocessed_groups.append((group, students[:student_count]))
            subject_groups[group] = subject_groups[group][student_count:]
    # Step 2: Initialize variables for the backtracking algorithm
    groups = [group for group, _ in preprocessed_groups]
    students = [students for _, students in preprocessed_groups]        
    best_solution = {"divisions": None, "deviation": float("inf")}
    recursive_calls = 0

    def calculate_hours(division):
        """Calculate total hours per subject in a division."""
        hours = {subject: 0 for subject in total_hours}
        for group, _ in division:
            for subject in group:
                hours[subject] = total_hours[subject]
        return hours

    def calculate_deviation(hours):
        """Calculate the total deviation of hours from the initial requirement."""
        return sum(abs(hours[sub] - total_hours[sub]) for sub in total_hours)

    def backtrack(current_divisions, remaining_groups, remaining_students):
        nonlocal recursive_calls
        recursive_calls += 1

        # Base case: no groups left to allocate
        if not remaining_groups:
            total_hours_all = {subject: 0 for subject in total_hours}
            for division in current_divisions:
                hours = calculate_hours(division)
                for subject in total_hours_all:
                    total_hours_all[subject] += hours[subject]
            deviation = calculate_deviation(total_hours_all)
            if deviation < best_solution["deviation"]:
                best_solution["divisions"] = [div[:] for div in current_divisions]
                best_solution["deviation"] = deviation
            return

        # Recursive case: allocate the next group
        group = remaining_groups[0]
        group_students = remaining_students[0]
        count = len(group_students)

        # Try allocating the group (or part of it) to each division
        for i in range(len(current_divisions)):
            division = current_divisions[i]
            division_student_count = sum(len(students) for _, students in division)

            if division_student_count + count <= max_students:
                # Allocate the entire group to this division
                division.append((group, group_students))
                backtrack(current_divisions, remaining_groups[1:], remaining_students[1:])
                division.pop()
            else:
                # Allocate a part of the group to this division
                split_count = max_students - division_student_count
                if split_count > 0:
                    division.append((group, group_students[:split_count]))
                    backtrack(
                        current_divisions,
                        [group] + remaining_groups[1:],
                        [group_students[split_count:]] + remaining_students[1:],
                    )
                    division.pop()

        # Start a new division with this group
        if len(current_divisions) < len(groups):
            current_divisions.append([(group, group_students)])
            backtrack(current_divisions, remaining_groups[1:], remaining_students[1:])
            current_divisions.pop()

    # Step 3: Initialize backtracking
    backtrack([], groups, students)

    # Log statistics
    print(f"Total Recursive Calls: {recursive_calls}")

    # Calculate the final subject hours for each subject
    total_hours_all_divisions = {subject: 0 for subject in total_hours}
    for division in best_solution["divisions"]:
        division_hours = calculate_hours(division)
        for subject in total_hours_all_divisions:
            total_hours_all_divisions[subject] += division_hours[subject]

    # Return the best solution with subject hours included
    return best_solution, total_hours_all_divisions


# Run the revised algorithm
result, subject_hours = allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students)
print("\nOptimal Division Allocation:")
if result["divisions"]:
    for i, division in enumerate(result["divisions"], start=1):        
        division_students = sum(len(students) for _, students in division)
        print(f"Division {i}: {division}, Total Students: {division_students}")
# # Print total deviation
# print("\nTotal Deviation:", result["deviation"])

# # Print the subject hours for each subject in the final solution
# print("\nFinal Subject Hours Across All Divisions:")
# for subject, hours in subject_hours.items():
#     print(f"{subject}: {hours} hours")

Total Recursive Calls: 10593

Optimal Division Allocation:
Division 1: [((<Subject: Compiler Design | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Mobile Computing and Wireless Communication | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Artificial Intelligence | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Information security | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Big Data Analytics | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>, <Subject: Mobile Application Development | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>), [<Student: 200280107141 | Anisio Alberto da Conceicao Jose Mandlate>, <Student: 200280107142 | Adilson Jossias A. Gafur>, <Student: 220283107003 | amansevak123@gmail.com>, <Student: 220283107011 | dipenmagdani@gmail.com>, <Student: 220283107019 | parmarharsh90

In [ ]:
# Constraints
min_students = 0  # No minimum size for divisions
max_students = 90  # Maximum size per division

def allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students):
    # Step 1: Preprocess subject groups to split any group exceeding max_students
    preprocessed_groups = []
    for group, count in subject_groups.items():
        while count > max_students:            
            preprocessed_groups.append((group, max_students))
            count -= max_students
        if count > 0:
            preprocessed_groups.append((group, count))        
    # Step 2: Initialize variables for the backtracking algorithm
    groups = [group for group, _ in preprocessed_groups]
    students = [count for _, count in preprocessed_groups]    
    best_solution = {"divisions": None, "deviation": float("inf")}
    recursive_calls = 0

    def calculate_hours(division):
        """Calculate total hours per subject in a division."""
        hours = {subject: 0 for subject in total_hours}
        for group, _ in division:
            for subject in group:
                hours[subject] = total_hours[subject]
        return hours

    def calculate_deviation(hours):
        """Calculate the total deviation of hours from the initial requirement."""
        return sum(abs(hours[sub] - total_hours[sub]) for sub in total_hours)

    def backtrack(current_divisions, remaining_groups, remaining_counts):
        nonlocal recursive_calls
        recursive_calls += 1

        # Base case: no groups left to allocate
        if not remaining_groups:            
            total_hours_all = {subject: 0 for subject in total_hours}
            for division in current_divisions:
                hours = calculate_hours(division)
                for subject in total_hours_all:
                    total_hours_all[subject] += hours[subject]
            deviation = calculate_deviation(total_hours_all)
            if deviation < best_solution["deviation"]:
                best_solution["divisions"] = [div[:] for div in current_divisions]
                best_solution["deviation"] = deviation
            return

        # Get the next group and its student count
        group = remaining_groups[0]
        count = remaining_counts[0]

        # Try allocating the group (or part of it) to each division
        for i in range(len(current_divisions)):
            division = current_divisions[i]
            division_student_count = sum(student_count for _, student_count in division)

            if division_student_count + count <= max_students:
                # Allocate the entire group to this division
                division.append((group, count))
                backtrack(
                    current_divisions,
                    remaining_groups[1:],
                    remaining_counts[1:],
                )
                # Backtrack
                division.pop()
            else:
                # Allocate a part of the group to this division
                split_count = max_students - division_student_count
                if split_count > 0:
                    # Add the split part to this division
                    division.append((group, split_count))
                    # Recurse with the remaining students of the group
                    backtrack(
                        current_divisions,
                        [group] + remaining_groups[1:],
                        [count - split_count] + remaining_counts[1:],
                    )
                    # Backtrack
                    division.pop()

        # Start a new division with this group
        if len(current_divisions) < len(groups):
            current_divisions.append([(group, count)])
            backtrack(
                current_divisions,
                remaining_groups[1:],
                remaining_counts[1:],
            )
            # Backtrack
            current_divisions.pop()

    # Step 3: Initialize backtracking
    backtrack([], groups, students)

    # Log statistics
    print(f"Total Recursive Calls: {recursive_calls}")

    # Calculate the final subject hours for each subject
    total_hours_all_divisions = {subject: 0 for subject in total_hours}
    for division in best_solution["divisions"]:
        division_hours = calculate_hours(division)
        for subject in total_hours_all_divisions:
            total_hours_all_divisions[subject] += division_hours[subject]

    # Return the best solution with subject hours included
    return best_solution, total_hours_all_divisions


# Run the revised algorithm
result, subject_hours = allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students)
division_objs = {}
print("\nOptimal Division Allocation:")
if result["divisions"]:
    for i, division in enumerate(result["divisions"], start=1):
        if f"div_{i}" not in division_objs:
            division_objs[f"div_{i}"] = []
            for group,count in division:
                division_objs[f"div_{i}"].append(subject_group_students[group][:count])
                subject_group_students[group] = subject_group_students[group][count:]
        division_students = sum(count for _, count in division)
        print(f"Division {i}: {division}, Total Students: {division_students}")

print("\nStudents in divisions:")
print(division_objs)
# Print total deviation
print("\nTotal Deviation:", result["deviation"])

# Print the subject hours for each subject in the final solution
print("\nFinal Subject Hours Across All Divisions:")
for subject, hours in subject_hours.items():
    print(f"{subject}: {hours} hours")